In [9]:
import pandas as pd
from datasets import Dataset

In [10]:
df = pd.read_feather("./data/processed/stt/wav2vec/test/test.feather")

In [11]:
Dataset.from_pandas(df)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 961
})